In [14]:
# detect.py - Local YOLOv5 version (after git clone)
import torch
import cv2
import streamlit as st
from PIL import Image
import numpy as np
import sys
import os

In [18]:
# Add local yolov5 path
YOLO_PATH = os.path.join(os.getcwd(), "yolov5")
sys.path.append(YOLO_PATH)

In [19]:
# Import from YOLOv5
from models.common import DetectMultiBackend
from utils.general import non_max_suppression, check_img_size
from utils.augmentations import letterbox
from utils.plots import Annotator
from utils.torch_utils import select_device

In [22]:
# Load model
weights = 'yolov5s.pt'
device = select_device('')
model = DetectMultiBackend(weights, device=device)
stride, names = model.stride, model.names
imgsz = check_img_size((640, 640), s=stride)

YOLOv5  2025-4-19 Python-3.10.0 torch-2.6.0+cpu CPU

100.0%

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


In [23]:
# Streamlit interface
st.title("Crop & Weed Detection System")
st.markdown("Upload an image to detect crops and weeds using YOLOv5.")


2025-04-19 10:40:11.618 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 10:40:11.767 
  command:

    streamlit run C:\Users\kaushal prasad\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-19 10:40:11.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 10:40:11.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 10:40:11.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [24]:
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

2025-04-19 10:40:23.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 10:40:23.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 10:40:23.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 10:40:23.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 10:40:23.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [25]:
if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    img0 = np.array(image)

    # Pre-process
    img = letterbox(img0, imgsz, stride=stride, auto=True)[0]
    img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW
    img = np.ascontiguousarray(img)

    img_tensor = torch.from_numpy(img).to(device)
    img_tensor = img_tensor.float() / 255.0
    if img_tensor.ndimension() == 3:
        img_tensor = img_tensor.unsqueeze(0)

    # Inference
    pred = model(img_tensor, augment=False, visualize=False)
    pred = non_max_suppression(pred, 0.25, 0.45, agnostic=False)

    # Annotate
    for i, det in enumerate(pred):
        annotator = Annotator(img0.copy(), line_width=2, example=str(names))
        if len(det):
            det[:, :4] = det[:, :4].round()
            for *xyxy, conf, cls in reversed(det):
                label = f'{names[int(cls)]} {conf:.2f}'
                annotator.box_label(xyxy, label, color=(0, 255, 0))

    st.image(annotator.result(), caption="Detection Result", use_column_width=True)